In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np

### Define the coupled derivative functions to integrate

Why is he making an array of y derivatives? How does this help us later?
Is it so they're in system of equations form?

In [ ]:
def dydx(x, y):
    
    # A second-order diff. eq. can be written in a system
    #of first-order diff. eqs. They're 'coupled'
    
    
    # Set an array of the derivative functions in the system
    y_derivs = np.zeros(2)
    
    # dydx = z
    y_derivs[0] =    y[1]  # Hence, y is an array
    
    # dzdx = -y
    y_derivs[1] = -1*y[0]

    return y_derivs

### Define the 4th order RK method

What is a core method?

### Declaration of Cask-Karp constants based on table

In [ ]:
a1, a2, a3, a4, a5, a6 = 0., 1/5, 3/10, 3/5, 1., 7/8
b21 = 1/5
b31, b32 = 3/40, 9/40
b41, b42, b43 = 3/10, -9/10, 6/5
b51, b52, b53, b54 = -11/54, 5/2, -70/27, 35/27
b61, b62, b63, b64, b65 = 1631/55296, 175/512, 575/13824, 44275/110592, 253/4096
c1, c2, c3, c4, c5, c6 = 37/378, 0., 250/621, 125/594, 0., 512/1771
c1s, c2s, c3s, c4s, c5s, c6s = 2825/27648, 0., 18575/48384, 13525/55296, 277/14336, 1/4

In [ ]:
def rk4_mv_core(dydx, xi, yi, nv, h):
    # h = stepsize. Distance between xi and yi
    # k[] declaration extracted from @zbriesem/stellarmodel/integration.py
    
    x = np.asarray(xi)
    y = np.asarray(yi)
    k = np.zeros((6, yi.shape[0]), dtype=float)
    
    # Starts with k1 = k[0]           What's args?
    k[0] = h * dydx(x + a1 * h, y)
    k[1] = h * dydx(x + a2 * h, y + b21 * k[0])
    k[2] = h * dydx(x + a3 * h, y + b31 * k[0] + b32 * k[1])
    k[3] = h * dydx(x + a4 * h, y + b41 * k[0] + b42 * k[1] + b43 * k[2])
    k[4] = h * dydx(x + a5 * h, y + b51 * k[0] + b52 * k[1] + b53 * k[2] + b54 * k[3])
    k[5] = h * dydx(x + a6 * h, y + b61 * k[0] + b62 * k[1] + b63 * k[2] + b64 * k[3] + b65 * k[4])
    
    
    # Advance y by a step h
    yipo = yi + c1*k[0] + c2*k[1] + c3*k[2] + c4*k[3] + c5*k[4] + c6*k[5]
    
    return yipo
    

Why doesn't he just define k_N for each order and then assign each a, b, b*, and c at the end? 

In that case, how would we make the iteration run through all the n?

I just really don't like that thing /6

### Adaptive step size driver

In [ ]:
def rk4_mv_ad(dydx, x_i, y_i, nv, h, tol):
    
    # Define safety scale
    SAFETY = 0.9     #This is the number we multiply by to reduce h
    H_NEW_FAC = 2.0  #?
    
    # Set a maximum number of iterations
    imax = 10000
    
    # Set an iteration variable
    i = 0
    
    # Create an error
    Delta = np.full(nv, 2*tol)
    """np.full returns an array with nv elements of value 2*tol
    Why would I want that?
    
    If nv was (a, b), it would make a matrix a x b
    
    Returns an ndarray"""
    
    # Rename the step
    h_step = h
    
    # Adjust step
    while (Delta.max()/tol > 1.0):
        
        # Estimate the error by taking one step of size h
        #vs. two steps of size h/2
        y_2 = rk4_mv_core(dydx, x_i, y_i, nv, h_step)
        y_1 = rk4_mv_core(dydx, x_i, y_i, nv, 0.5*h_step)
        y_11 = rk4_mv_core(dydx, x_i+0.5*h_step, y_1, nv, 0.5*h_step)    #TYPO _step FIXED
                                                                         #TYPO _1 FIXED
        # Compute error
        Delta = np.fabs(y_2 - y_11)
        
        # If the error is too large, take a smaller step
        if (Delta.max()/tol > 1.0):
            h_step *= SAFETY * (Delta.max()/tol)**(-0.25)
        
        # Check iteration
        if (i>= imax):
            print ("Too many iterations in rk4_mv_ad()")
            raise StopIteration("Ending after i = ", i)
            
        i += 1
        
    # Why do we want to take a bigger step? This makes no sense to me..
    h_new = np.fmin(h_step * (Delta.max()/tol)**(-0.9), h_step*H_NEW_FAC)
    
    return y_2, h_new, h_step

### Define a wrapper for RK4

In [ ]:
def rk4_mv(dfdx, a, b, y_a, tol):
    #y_a arange of initial values for each diff eq in the couple
    
    # Define our starting step
    xi = a
    yi = y_a.copy()
    
    # An initial step size == make vary small
    h = 1.0e-4 * (b-a)  #Why so small? I thought h was just b-a?
    
    # Set a maximum number of iterations
    imax = 10000
    
    # Set an iteration variable
    i = 0
    
  #What is all this for?:  
    # Set the number of coupled ODES to the size of y_a
    nv = len(y_a)
    
    # Set the initial conditions
    x = np.full(1, a)
    y = np.full((1,nv), y_a)
    
    # Set a flag
    flag = 1
    
    while(flag):
        #Calculate y_i+1:
        yi_new, h_new, h_step = rk4_mv_ad(dydx, xi, yi, nv, h, tol)
        
        # Update the step
        h = h_new
        
        if(xi+h_step > b):
            
            # Take a smaller step
            h = b-xi
            
            # Recalculate y_i+1
            yi_new, h_new, h_step = rk4_mv_ad(dydx, xi, yi, nv, h, tol)
            
            # Break
            flag = 0
            
        # Update values
        xi += h_step
        yi[:] = yi_new[:]
        
        # Add the step to the arrays
        x = np.append(x, xi)
        y_new = np.zeros((len(x), nv))
        y_new[0:len(x)-1, :] = y
        y_new[-1,:] = yi[:]
        del y
        y = y_new
        
        # Prevent too many iteratons
        if(i >= imax):
            
            print( "Maximum iterations reached.")
            raise StopIteration("Iteration number = ", i)
            
        i +=1
        
        # Output some information
        s = "i = %3d\tx = %9.8f\th = %9.8f\tb=%9.8f" % (i, xi, h_step, b)
        print(s)
        
        # Break if new xi is == b
        if (xi == b):
            flag = 0
            
    return x, y            

### Perform the integration

In [ ]:
a = 0.0
b = 2.0 * np.pi

y_0 = np.zeros(2)
y_0[0] = 0.0
y_0[1] = 1.0
nv = 2

tolerance = 1.0e-6

#Perform the integration
x, y = rk4_mv(dydx, a, b, y_0, tolerance)

### Plot the result

In [ ]:
plt.plot(x, y[:,0], 'o', label='y(x)')
plt.plot(x, y[:,1], 'o', label='dydx(x)')
xx = np.linspace(0, 2.0*np.pi, 1000)
plt.plot(xx, np.sin(xx), label='sin(x)')
plt.plot(xx, np.cos(xx), label='cos(x)')
plt.xlabel('x')
plt.ylabel('y, dy/dx')
plt.legend(frameon=False)

In [ ]:
sine = np.sin(x)
cosine = np.cos(x)

y_error = (y[:,0] - sine)
dydx_error = (y[:,1] - cosine) ##TYPO comma(,) FIXED

plt.plot(x, y_error, label='y(x) Error')
plt.plot(x, dydx_error, label="dydx(x) Error") ##TYPO comma(,) FIXED
plt.legend(frameon = False)